data set class defined

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io


class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None
                 ):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[idx, 1]))
        if self.transform:
            image = self.transform(image)

        return image, y_label

    


zip file extraction of dog vs cat data set

In [ ]:
import zipfile
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

In [ ]:
import matplotlib.pyplot as plt

def loss_plot(epochs, loss):
    plt.plot(epochs, loss)

googlenet model usage 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# hyperparameter for rnn
# inputsize means depth or features
learning_rate = 0.001
batch_size = 16
num_epochs = 1


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()


dataset = CatsAndDogsDataset(csv_file="../input/catvsdog/catvsdog.csv", root_dir='./train',
                             transform=transforms.Compose([transforms.ToTensor(),transforms.Resize([100, 100]) ]))
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# initialize network
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    print("epoch",epoch)
    for batch_index, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        # print(data.shape)
        scores = model(data)
        loss = criterion(scores, targets)
        
        if batch_index % 10 == 0:
            print(scores.shape)
        
        # backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    print("loss = ", loss)


def check_accuracy(loader, model,train=1):
    if train:
        print("training model")
    else:
        print("testing model")

    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print("accuracy = ", (float(num_correct) / float(num_samples)) * 100)
    model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model,0)


basic NN model implementation

In [ ]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

input_size = 100
num_classes = 2
learning_rate = 0.001
batch_size = 128
num_epoch = 2

dataset = CatsAndDogsDataset(csv_file="../input/catvsdog/catvsdog.csv", root_dir='./train',
                             transform=transforms.Compose([transforms.ToTensor(), transforms.Resize([100, 100])]))
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)


class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size*300, 500)
        self.fc2 = nn.Linear(500, 50)
        self.fc3 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = NN(input_size, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epoch):
    losses=[]
    batch_idxs=[]
    for batch_idx, (data, targets) in enumerate(train_loader):
#         print(batch_idx)
        data = data.to(device=device)
        targets = targets.to(device=device)

        data = data.reshape(data.shape[0], -1)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        if batch_idx % 10 ==0:
            print(loss,score)

        # backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        
#     loss_plot(batch_idxs,losses)


accuracy check for nn model

In [ ]:

def check_accuracy(loader, model):
    num_correct = 0
    total_sample = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            print(scores)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            total_sample += predictions.size(0)

        print("accuracy = ", (float(num_correct) / float(total_sample)) * 100)
    model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

CNN model implementation

In [ ]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

in_channels = 3
num_classes = 2
learning_rate = 0.0001
batch_size = 64
num_epoch = 5

class CNN(nn.Module):
    def __init__(self, in_channles, num_classes):
        super(CNN, self).__init__()
        self.conv6 = nn.Conv2d(in_channels=in_channles, out_channels=6, kernel_size=(3, 3), stride=(1, 1),
                               padding=(1, 1))
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(in_channels=6, out_channels=8, kernel_size=(3, 3), stride=(2, 2),
                               padding=(0, 0))
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=17, kernel_size=(3, 3), stride=(1, 1),
                               padding=(0, 0))
        self.pool3 = nn.MaxPool2d(2, stride=2)
        self.fc1 = nn.Linear(5 * 5 * 17, num_classes)

    def forward(self, x):
        x = F.relu(self.conv6(x))
#         print(x.shape)
        x = self.pool1(x)
#         print(x.shape)
        x = F.relu(self.conv1(x))
#         print(x.shape)
        x = self.pool2(x)
#         print(x.shape)
        x = F.relu(self.conv2(x))
#         print(x.shape)
        x = self.pool3(x)
#         print(x.shape)
#         x = F.relu(self.conv3(x))
# #         print(x.shape)
#         x = F.relu(self.conv4(x))
# #         print(x.shape)
#         x = F.relu(self.conv5(x))
#         print(x.shape)
        x = x.reshape(x.shape[0], -1)
        x = F.softmax(self.fc1(x),dim=1)
        return x
    

dataset = CatsAndDogsDataset(csv_file="../input/catvsdog/catvsdog.csv", root_dir='./train',
                             transform=transforms.Compose([transforms.ToTensor(), transforms.Resize([101, 101])]))
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

model = CNN(in_channels, num_classes).to(device)
# model = torchvision.models.vgg16(pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

CNN model training

In [ ]:
for epoch in range(num_epoch):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        if batch_idx%100==0:
            print(loss)

        # backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

CNN model accuracy check

In [ ]:

def check_accuracy(loader, model):
    num_correct = 0
    total_sample = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            total_sample += predictions.size(0)

        print("accuracy = ", (float(num_correct) / float(total_sample)) * 100)
    model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

In [ ]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

input_size = 100
num_layers=2
sequence_length=100
hidden_nodes = 100
num_classes = 2
learning_rate = 0.001
batch_size = 128
num_epoch = 5

class RNN(nn.Module):
    def __init__(self, input_size, hidden_nodes,num_layers,num_classes):
        super(RNN, self).__init__()
        self.hidden_size=hidden_nodes
        self.num_layers=num_layers
        self.rnn=nn.RNN(input_size,hidden_nodes,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_nodes * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
        x,_=self.rnn(x,h0)
        x=x.reshape(x.shape[0],-1)
        x = self.fc(x)
        
        return x
    

dataset = CatsAndDogsDataset(csv_file="../input/catvsdog/catvsdog.csv", root_dir='./train',
                             transform=transforms.Compose([transforms.ToTensor(), transforms.Resize([100, 100])]))
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)


model = RNN(input_size, hidden_nodes,num_layers,num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epoch):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data[:,-1,:,:]
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        if batch_idx%100==0:
            print(loss)

        # backward
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

In [ ]:

def check_accuracy(loader, model):
    num_correct = 0
    total_sample = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x=x[:,-1,:,:]
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            total_sample += predictions.size(0)

        print("accuracy = ", (float(num_correct) / float(total_sample)) * 100)
    model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)